In [ ]:
#@title Installation
!pip install timm foolbox

     |████████████████████████████████| 296kB 5.8MB/s 
     |████████████████████████████████| 1.7MB 6.3MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 163kB 19.3MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
#@title Mount Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#@title Imports

import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import L2PGD

import timm

from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

from tqdm import tqdm

import cv2

import matplotlib.pyplot as plt

PyTorch Version:  1.8.1+cu101
Torchvision Version:  0.9.1+cu101


In [ ]:
#@title Global Variables

data_dir = "/content/drive/MyDrive/598dataset/tinyImageNet/TinyImageNet/"
generated_dir = "/content/drive/MyDrive/598dataset/detector_pgd_2/"
#generated_dir = "/content/drive/MyDrive/598dataset/detector_pgd_5/"
#generated_dir = "/content/drive/MyDrive/598dataset/detector_pgd_10/"

# Number of classes in the dataset
num_classes = 16

# Batch size for training (change depending on how much memory you have)
batch_size = 32

input_size = 224

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#@title timm model creation
model_ft = timm.create_model('vit_base_patch16_224', pretrained=True)
num_ftrs = model_ft.head.in_features
model_ft.head = nn.Linear(num_ftrs, num_classes)

model_ft.load_state_dict(torch.load('/content/drive/MyDrive/598dataset/ViTbase_16_224_tiny_15classes.pth'))
model_ft = model_ft.to(device)
model_ft.eval()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth" to /root/.cache/torch/hub/checkpoints/jx_vit_base_p16_224-80ecf9dd.pth


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (

In [ ]:
#@title foolbox model def
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = PyTorchModel(model_ft, bounds=(0, 1), preprocessing=preprocessing)

In [ ]:
#@title data loader init
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, 'tiny_'+x),
                                          transform[x])
                  for x in ['train', 'val']}
print(len(image_datasets['train'].classes))                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=False, num_workers=0, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(class_names)

16
['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6', '7', '8', '9']


In [ ]:
#@title attack def

attack = L2PGD(rel_stepsize=0.8, steps=2)
#attack = L2PGD(rel_stepsize=0.3, steps=5)
#attack = L2PGD(rel_stepsize=0.15, steps=10)

In [ ]:
#@title perform attack and save train image
epsilons = [
    0.5,
]
count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
train_storage_path = os.path.join(generated_dir,'train/adv')
with tqdm(dataloaders['train'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
    for i in clipped_advs:
      for j in range(batch_size):
        torchvision.utils.save_image(i[j,...], os.path.join(train_storage_path,str(count) + '.png'))
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"  L2 norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

100%|██████████| 500/500 [1:25:38<00:00, 10.28s/batch]

  L2 norm ≤ 0.5   : 36.6 %


In [ ]:
#@title perform attack and save test image
epsilons = [
    0.5,
]
count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
train_storage_path = os.path.join(generated_dir,'val/adv')
clipped_advs_list_val = []
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
    for i in clipped_advs:
      for j in range(batch_size):
        torchvision.utils.save_image(i[j,...], os.path.join(train_storage_path,str(count) + '.png'))
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"  L2 norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

100%|██████████| 50/50 [10:02<00:00, 12.04s/batch]

  L2 norm ≤ 0.5   : 49.8 %


In [ ]:
#@title PGD5
generated_dir = "/content/drive/MyDrive/598dataset/detector_pgd_5/"
attack = L2PGD(rel_stepsize=0.3, steps=5)
epsilons = [
    0.5,
]
count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
train_storage_path = os.path.join(generated_dir,'train/adv')
with tqdm(dataloaders['train'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
    for i in clipped_advs:
      for j in range(batch_size):
        torchvision.utils.save_image(i[j,...], os.path.join(train_storage_path,str(count) + '.png'))
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"  L2 norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

epsilons = [
    0.5,
]
count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
train_storage_path = os.path.join(generated_dir,'val/adv')
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
    for i in clipped_advs:
      for j in range(batch_size):
        torchvision.utils.save_image(i[j,...], os.path.join(train_storage_path,str(count) + '.png'))
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"  L2 norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]

  L2 norm ≤ 0.5   : 28.6 %


100%|██████████| 50/50 [03:20<00:00,  4.00s/batch]

  L2 norm ≤ 0.5   : 40.6 %


In [ ]:
#@title PGD10
generated_dir = "/content/drive/MyDrive/598dataset/detector_pgd_10/"
attack = L2PGD(rel_stepsize=0.15, steps=10)
epsilons = [
    0.5,
]
count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
train_storage_path = os.path.join(generated_dir,'train/adv')
with tqdm(dataloaders['train'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
    for i in clipped_advs:
      for j in range(batch_size):
        torchvision.utils.save_image(i[j,...], os.path.join(train_storage_path,str(count) + '.png'))
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"  L2 norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

epsilons = [
    0.5,
]
count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
train_storage_path = os.path.join(generated_dir,'val/adv')
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
    for i in clipped_advs:
      for j in range(batch_size):
        torchvision.utils.save_image(i[j,...], os.path.join(train_storage_path,str(count) + '.png'))
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"  L2 norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]

  L2 norm ≤ 0.5   : 25.6 %


100%|██████████| 50/50 [16:54<00:00, 20.29s/batch]

  L2 norm ≤ 0.5   : 36.7 %


In [ ]:
#@title pgd 1 L inf ep=0.01
from foolbox.attacks import LinfPGD

generated_dir = "/content/drive/MyDrive/598dataset/detector_pgd_1_inf_0.01/"
attack = LinfPGD(rel_stepsize=1.5, steps=1)
epsilons = [
    0.01,
]
count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
train_storage_path = os.path.join(generated_dir,'train/adv')
with tqdm(dataloaders['train'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
    for i in clipped_advs:
      for j in range(batch_size):
        torchvision.utils.save_image(i[j,...], os.path.join(train_storage_path,str(count) + '.png'))
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"  L2 norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

epsilons = [
    0.01,
]
count = 0
running_acc = torch.zeros(len(epsilons)).to(device)
train_storage_path = os.path.join(generated_dir,'val/adv')
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)
    for i in clipped_advs:
      for j in range(batch_size):
        torchvision.utils.save_image(i[j,...], os.path.join(train_storage_path,str(count) + '.png'))
    running_acc += 1 - success.float().mean(axis=-1)
    count += 1
running_acc /= count
for eps, acc in zip(epsilons, running_acc):
    print(f"  L2 norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

  0%|          | 0/50 [00:00<?, ?batch/s]

  L2 norm ≤ 0.01  : 21.7 %


100%|██████████| 50/50 [01:06<00:00,  1.33s/batch]

  L2 norm ≤ 0.01  : 29.0 %
